# i) Regra da soma

In [4]:
import os
import numpy as np
import pandas as pd
import clustering

## Datasets

In [5]:
DATA_BASE_PATH = "./data"

FAC_FILE = os.path.join(DATA_BASE_PATH, "mfeat-fac")
FOU_FILE = os.path.join(DATA_BASE_PATH, "mfeat-fou")
KAR_FILE = os.path.join(DATA_BASE_PATH, "mfeat-kar")

fac = np.loadtxt(FAC_FILE, dtype=int)
fou = np.loadtxt(FOU_FILE, dtype=float)
kar = np.loadtxt(KAR_FILE, dtype=float)

## Importando melhor resultado

In [6]:
best_result = clustering.import_best_result("data/melhor_resultado_todas.pickle")
partition, y_true = clustering.get_hard_patitions(best_result["membership_degree"])


## Probabilidade à priori das classes

In [7]:
Pw = np.array([len(c)/2000 for c in partition])
Pw

array([0.1175, 0.116 , 0.1355, 0.1465, 0.0405, 0.01  , 0.1305, 0.151 ,
       0.0815, 0.071 ])

## Função de densidade

In [29]:
# def calc_gaussian_density_prob(xk, d, means, var, cov_matrix):
#     coef = np.power(2*np.pi, -d/2) 
#     inv_cov_matrix = np.linalg.inv(cov_matrix)
#     (sign, logdet) = np.linalg.slogdet(inv_cov_matrix)
#     sqrt_det_inv_cov = np.sqrt(sign*np.exp(logdet))
#     diff = xk - means
#     exp_exp = np.dot((-1/2)*np.dot(diff.T, inv_cov_matrix), diff)
#     exp_func = np.exp(exp_exp)
    
#     return coef * sqrt_det_inv_cov * exp_func

In [30]:

# def calc_gaussian_bayesian_data(x, partition):
#     n,d = x.shape
#     qtd_w = len(partition)
#     means = np.array([x[idxs].mean(axis=0) for idxs in partition])
#     var = np.array([((x[idxs]-means[i])**2).mean(axis=0) for i, idxs in enumerate(partition)])
#     cov_matrix = [np.zeros((d,d)) for _ in range(qtd_w)]

#     for i in range(qtd_w):
#         np.fill_diagonal(cov_matrix[i], var[i])
                
#     p_x_w = np.empty((n, qtd_w))
    
#     for k in range(n):
#         for i in range(qtd_w):
#             p_x_w[k, i] = calc_gaussian_density_prob(x[k], d, means[i], var[i], cov_matrix[i])   
    
#     return p_x_w

def calc_prob_posteriori(p_x_w, Pw):
    qtd_x, qtd_w = p_x_w.shape 
    p_w_x = np.empty((qtd_x, qtd_w))
    
    for k in range(qtd_x):
        for i in range(qtd_w):
            sum_all = np.dot(p_x_w[k], Pw)
            p_w_x[k,i] = (p_x_w[k,i] * Pw[i])/sum_all
       
    return p_w_x


## Densidades por dataset

In [31]:
# fac_byn_density_probs = calc_gaussian_bayesian_data(fac, partition)
# fou_byn_density_probs = calc_gaussian_bayesian_data(fou, partition)
# kar_byn_density_probs = calc_gaussian_bayesian_data(kar, partition)

In [32]:
# fac_byn_density_probs[0,0], fou_byn_density_probs[0,0], kar_byn_density_probs[0,0]

## Prob. à posteriori por view

In [33]:
# fac_byn_posteriori_probs = calc_prob_posteriori(fac_byn_density_probs, Pw)
# fou_byn_posteriori_probs = calc_prob_posteriori(fou_byn_density_probs, Pw)
# kar_byn_posteriori_probs = calc_prob_posteriori(kar_byn_density_probs, Pw)


## Regra da soma

Precisei tirar o fac porque seus valores são nulos. TENTAR CORRIGIR ISSO

In [34]:
def regra_soma_padrao(views, Pw):
    qtd_x = views[0].shape[0]
    qtd_w = len(Pw)
    x_sum_w = np.empty((qtd_x, qtd_w))

    for i in range(qtd_x):
        for k in range(qtd_w):
            views_sum = sum([v[i,k] for v in views])
            x_sum_w[i,k] = (1-len(views))*Pw[k] + views_sum
                    
    y_pred = x_sum_w.argmax(axis = 1) 
    return y_pred

## Estimador do scikit

In [35]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels

class ClassificaforBayesiano(BaseEstimator, ClassifierMixin):
    def __init__(self, partition, Pw):
        self.partition = partition
        self.Pw = Pw
    
    def fit(self, X, y):
        X, y = check_X_y(X, y)
         
        self.classes_ = unique_labels(y)
        self._fit_gaussian_bayesian_data(X)
        self.X_ = X
        self.y_ = y
        return self
    
    def _fit_gaussian_bayesian_data(self, X):
        n, d = X.shape
        qtd_w = len(self.partition)
        self.means = np.array([X[idxs].mean(axis=0) for idxs in self.partition])
        self.var = np.array([((X[idxs]-self.means[i])**2).mean(axis=0) for i, idxs in enumerate(self.partition)])
        self.cov_matrix = [np.zeros((d,d)) for _ in range(qtd_w)]

        for i in range(qtd_w):
            np.fill_diagonal(self.cov_matrix[i], self.var[i])

        return self
    
    def _calc_gaussian_density_prob(self, xk, cls):
        d = xk.shape[0]
        coef = np.power(2*np.pi, -d/2)
        inv_cov_matrix = np.linalg.inv(self.cov_matrix[cls])
        (sign, logdet) = np.linalg.slogdet(inv_cov_matrix)
        sqrt_det_inv_cov = np.sqrt(sign*np.exp(logdet))
        diff = xk - self.means[cls]
        exp_exp = np.dot((-1/2)*np.dot(diff.T, inv_cov_matrix), diff)
        exp_func = np.exp(exp_exp)

        return coef * sqrt_det_inv_cov * exp_func

    def predict_proba(self, X):
        check_is_fitted(self)
        X = check_array(X)
        
        desity_probs = np.empty((X.shape[0], len(self.classes_)))
        for k in range(desity_probs.shape[0]):
            for j in range(len(self.classes_)):
                desity_probs[k,j] = self._calc_gaussian_density_prob(X[k], j)
        
        post_probs = calc_prob_posteriori(desity_probs, self.Pw)
        
        return post_probs
        
        

In [36]:
class RegraSomaClasificadorBayesiano(BaseEstimator, ClassifierMixin):
    def __init__(self, partition, Pw):
#         self.views = views
        self.partition = partition
        self.Pw = Pw
        self.clfs = []        
    
    def fit(self, X, y):
#         X, y = check_X_y(X, y)
         
        self.classes_ = unique_labels(y)
        self.X_ = X
        self.y_ = y
        
        for x in X:
            clf = ClassificaforBayesiano(self.partition, Pw)
            clf.fit(x,y)
            self.clfs.append(clf)
            
        return self
    
    def predict(self, X):
        assert len(X) == len(self.clfs)
        # Check is fit had been called
        check_is_fitted(self)

        # Input validation
#         X = check_array(X)
        
        post_probs = [clf.predict_proba(x) for clf, x in zip(self.clfs, X)]
        
        return self.regra_soma(post_probs, Pw=self.Pw)
    
    def get_params(self, deep=True):
        return {"Pw": self.Pw, "partition": self.partition}
    
    def regra_soma(self, matrizes, Pw):
        qtd_x = matrizes[0].shape[0]
        qtd_w = len(Pw)
        x_sum_w = np.empty((qtd_x, qtd_w))
        
        for i in range(qtd_x):
            for k in range(qtd_w):
                views_sum = sum([v[i,k] for v in matrizes])
                # views_sum =  fac_p_w_x[i,k] fou_p_w_x[i,k] + kar_p_w_x[i,k]
                x_sum_w[i,k] = (1-len(matrizes))*Pw[k] + views_sum
                
        y_pred = x_sum_w.argmax(axis = 1) 
        return y_pred

# def regra_soma_padrao(fac_p_w_x, fou_p_w_x, kar_p_w_x, Pw):
# x_sum_w = np.empty((10, 2000))

# for i in range(10):
#     for k in range(2000):
#         views_sum =  fou_p_w_x[i,k] + kar_p_w_x[i,k]
#         # views_sum =  fac_p_w_x[i,k] fou_p_w_x[i,k] + kar_p_w_x[i,k]
#         x_sum_w[i,k] = (1-2)*Pw[i] + views_sum

# y_pred = x_sum_w.argmax(axis = 0) 
# return y_pred

## Validação cruzada

- **Observação**: No modelo gaussiano,  as desidades de probabilidades da VIEW1 (mfeat-fac) são todas zeradas, o que causa uma baixa na performance geral do modelo da regra da soma. Por esta razão, apenas neste modelo, ela foi desconsiderada. Portanto, no modelo gaussiano apresentamos os resultados os dados considerando somente VIEW2 e VIEW3.

- As médias das métricas para cada split da validação estão no arquivo **data/gaussian_training_data.csv**. Nota-se que muitos resultados aprensetam valores altos ou representações do infinito ("inf"). Isso se deu por conta de problemas de overflow na multiplicação usando valores muito pequenos fato que, aparentemente, foi causado pela quantidade de splits.

In [61]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import precision_recall_fscore_support, accuracy_score


RANDOM_SEED = 42
FOLDS = 10
REPEATS = 30
REAPEATS_GAUSSIAN = 10

def get_splited_partition(idxs, y_true):
    partition = [[] for i in range(10)]
    
    for i,indice in enumerate(idxs):
        partition[y_true[indice]].append(i)
        
    return partition
    
cv = RepeatedStratifiedKFold(FOLDS, REPEATS, RANDOM_SEED)

train_precision_scores = np.empty((cv.get_n_splits(),))
train_recall_scores = np.empty((cv.get_n_splits(),))
train_f1_scores = np.empty((cv.get_n_splits(),))
train_acc_scores = np.empty((cv.get_n_splits(),))

test_precision_scores = np.empty((cv.get_n_splits(),))
test_recall_scores = np.empty((cv.get_n_splits(),))
test_f1_scores = np.empty((cv.get_n_splits(),))
test_acc_scores = np.empty((cv.get_n_splits(),))

#views = [fac, fou, kar]
views = [fou, kar]

for i, (train_index, test_index) in enumerate(cv.split(fac, y_true)):
    print(f"{i+1}/{cv.get_n_splits()}", end=" ")
    
    test_views = [v[test_index] for v in views]
    train_views = [v[train_index] for v in views]
    
    local_partition = get_splited_partition(train_index, y_true)
    clf = RegraSomaClasificadorBayesiano(local_partition, Pw)
    
    clf.fit(train_views, y_true[train_index])
    
    y_pred_train = clf.predict(train_views)
    y_pred_test = clf.predict(test_views)
    
    scores_train = precision_recall_fscore_support(y_true[train_index], 
                                                       y_pred_train, 
                                                       average="macro")

    train_precision_scores[i] = scores_train[0]
    train_recall_scores[i] = scores_train[1]
    train_f1_scores[i] = scores_train[2]
    train_acc_scores[i] = accuracy_score(y_true[train_index], y_pred_train)

    scores_test = precision_recall_fscore_support(y_true[test_index], 
                                                   y_pred_test, 
                                                   average="macro")

    test_precision_scores[i] = scores_test[0]
    test_recall_scores[i] = scores_test[1]
    test_f1_scores[i] = scores_test[2]
    test_acc_scores[i] = accuracy_score(y_true[test_index], y_pred_test)

    report = dict(
        runs = cv.get_n_splits(),
        split=i+1,
        train_accuracy = train_acc_scores[i],
        train_precision = train_precision_scores[i],
        train_recall = train_recall_scores[i],
        train_fscore = train_f1_scores[i],
        test_accuracy = test_acc_scores[i],
        test_precision = test_precision_scores[i],
        test_recall = test_recall_scores[i],
        test_fscore = test_f1_scores[i],

    )

    if os.path.exists("data/gaussian_300_splits.csv"):
        pd.DataFrame([report]).to_csv("data/gaussian_300_splits.csv", mode="a", header=False, decimal=",", index=False)
    else:
        pd.DataFrame([report]).to_csv("data/gaussian_300_splits.csv", mode="a", header=True, decimal=",", index=False)



/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass n_splits=10, n_repeats=30, random_state=42 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


1/300 2/300 3/300 4/300 5/300 6/300 7/300 8/300 9/300 10/300 11/300 12/300 13/300 14/300 15/300 16/300 17/300 18/300 19/300 20/300 21/300 22/300 23/300 24/300 25/300 26/300 27/300 28/300 29/300 30/300 31/300 32/300 33/300 34/300 35/300 36/300 37/300 38/300 39/300 40/300 41/300 42/300 43/300 44/300 45/300 46/300 47/300 48/300 49/300 50/300 51/300 52/300 53/300 54/300 55/300 56/300 57/300 58/300 59/300 60/300 61/300 62/300 63/300 64/300 65/300 66/300 67/300 68/300 69/300 70/300 71/300 72/300 73/300 74/300 75/300 76/300 77/300 78/300 79/300 80/300 81/300 82/300 83/300 84/300 85/300 86/300 87/300 88/300 89/300 90/300 91/300 92/300 93/300 94/300 95/300 96/300 97/300 98/300 99/300 100/300 101/300 102/300 103/300 104/300 105/300 106/300 107/300 108/300 109/300 110/300 111/300 112/300 113/300 114/300 115/300 116/300 117/300 118/300 119/300 120/300 121/300 122/300 123/300 124/300 125/300 126/300 127/300 128/300 129/300 130/300 131/300 132/300 133/300 134/300 135/300 136/300 137/300 138/300 139/

/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


214/300 215/300 216/300 217/300 218/300 219/300 220/300 221/300 222/300 223/300 224/300 225/300 226/300 227/300 228/300 229/300 230/300 231/300 232/300 233/300 234/300 235/300 236/300 237/300 238/300 239/300 240/300 241/300 242/300 243/300 244/300 245/300 246/300 247/300 248/300 249/300 250/300 251/300 252/300 253/300 254/300 255/300 256/300 257/300 258/300 259/300 260/300 261/300 262/300 263/300 264/300 265/300 266/300 267/300 268/300 269/300 270/300 271/300 272/300 273/300 274/300 275/300 276/300 277/300 278/300 279/300 280/300 281/300 282/300 283/300 284/300 285/300 286/300 287/300 288/300 289/300 290/300 291/300 292/300 293/300 294/300 295/300 296/300 297/300 298/300 299/300 300/300 

## Treinamento com base completa

- O objetivo aqui é determinar o desempenho do classificador por classe, tendo em vista diversas métricas de classificação. Observar os resultados no arquivo **data/gaussian_classification_report.txt**

In [38]:
from sklearn.metrics import classification_report

#views = [fac, fou, kar]
views = [fou, kar]

clf = RegraSomaClasificadorBayesiano(partition, Pw)

clf.fit(views, y_true)

y_pred = clf.predict(views)

report = classification_report(y_true, y_pred, output_dict=False, digits=4)

acc_score_gaussian = accuracy_score(y_true, y_pred)

with open("data/gaussian_classification_report.txt", "w") as report_file:
    report_file.write(report)


## Estimativa pontual e intervalo de confiança

In [39]:
p = acc_score_gaussian
z_padrao = 1.96 # Confiança de 95%

def calc_intervalo_confiança(p, z=z_padrao, n=2000):
    diff = z*np.sqrt(p*(1-p)/n)
    return (round(p - diff, 4), round(p+diff, 4))

print("Estimativa pontual: ", p)
print("Intervalo de confiança: ", calc_intervalo_confiança(p))

Estimativa pontual:  0.7555
Intervalo de confiança:  (0.7367, 0.7743)


# ii) K-Vizinhos

## Normalizando 


In [40]:
from sklearn.preprocessing import minmax_scale

fac_norm = minmax_scale(fac)
fou_norm = minmax_scale(fou)
kar_norm = minmax_scale(kar)

## Distâncias entre os elementos

In [41]:
from sklearn.metrics.pairwise import euclidean_distances

fac_dist = euclidean_distances(fac_norm, fac_norm)
fou_dist = euclidean_distances(fou_norm, fou_norm)
kar_dist = euclidean_distances(kar_norm, kar_norm)

## Função de densidade

In [42]:
def calc_knn_density_prob(view_dists, k, Pw, y_true):
    qtd_x = view_dists.shape[0]
    qtd_w = len(Pw)
    
    p_x_w = np.empty((qtd_x, qtd_w))
    k_vizinhos = view_dists.argsort(axis=1)[:,:k+1]
    
    for j in range(qtd_x):
        w_vizinhos = y_true[k_vizinhos[j, 1:]]
        for i in range(qtd_w):
            p_x_w[j,i] = (w_vizinhos == i).sum()/k
                    
    return p_x_w
    

## Desidades por dataset

In [43]:
# fac_knn_density_probs = calc_knn_density_prob(fac_dist, 5, Pw, y_true)
# fou_knn_density_probs = calc_knn_density_prob(fou_dist, 5, Pw, y_true)
# kar_knn_density_probs = calc_knn_density_prob(kar_dist, 5, Pw, y_true)


## Prob. à posteriori por view

In [44]:
# fac_knn_posteriori_probs = calc_prob_posteriori(fac_knn_density_probs, Pw)
# fou_knn_posteriori_probs = calc_prob_posteriori(fou_knn_density_probs, Pw)
# kar_knn_posteriori_probs = calc_prob_posteriori(kar_knn_density_probs, Pw)

## Regra da soma

In [45]:
# y_pred_knn_all = regra_soma_padrao([fac_knn_posteriori_probs, 
#                                    fou_knn_posteriori_probs, 
#                                    kar_knn_posteriori_probs], Pw)

# print("Acurácia: ", accuracy_score(y_true, y_pred_knn_all))
# print("Medida-F: ", f1_score(y_true, y_pred_knn_all, average="macro"))

## Validação cruzada

In [46]:
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd
import os

cv = RepeatedStratifiedKFold(FOLDS, REPEATS, RANDOM_SEED)


k_range = range(3,4)
views = [fac, fou, kar]

for k in k_range:
    
    train_precision_scores = np.empty((cv.get_n_splits(),))
    train_recall_scores = np.empty((cv.get_n_splits(),))
    train_f1_scores = np.empty((cv.get_n_splits(),))

    test_precision_scores = np.empty((cv.get_n_splits(),))
    test_recall_scores = np.empty((cv.get_n_splits(),))
    test_f1_scores = np.empty((cv.get_n_splits(),))
    
    for i, (train_index, test_index) in enumerate(cv.split(fac, y_true)):
        print(f"[k={k}] {i+1}/{cv.get_n_splits()}", end=" ")

        train_views = [minmax_scale(v[train_index]) for v in views]
        test_views = [minmax_scale(v[test_index]) for v in views]

        train_views_dists = [euclidean_distances(v,v) for v in train_views]
        test_views_dists = [euclidean_distances(v,v) for v in test_views]

    #     y_true_split = get_splited_partition(train_index, y_true)

        train_views_density_probs = [calc_knn_density_prob(v, k, Pw, y_true[train_index]) 
                                     for v in train_views_dists]

        train_views_post_probs = [calc_prob_posteriori(v, Pw) 
                                  for v in train_views_density_probs]

        y_pred_train = regra_soma_padrao(train_views_post_probs, Pw)


        test_views_density_probs = [calc_knn_density_prob(v, k, Pw, y_true[test_index]) 
                                     for v in test_views_dists]

        test_views_post_probs = [calc_prob_posteriori(v, Pw) 
                                  for v in test_views_density_probs]

        y_pred_test = regra_soma_padrao(test_views_post_probs, Pw)
        
        scores_train = precision_recall_fscore_support(y_true[train_index], 
                                                       y_pred_train, 
                                                       average="macro")
        
#         train_precision_scores[i] = scores_train[0]
#         train_recall_scores[i] = scores_train[1]
#         train_f1_scores[i] = scores_train[2]

#         scores_test = precision_recall_fscore_support(y_true[test_index], 
#                                                        y_pred_test, 
#                                                        average="macro")
        
#         test_precision_scores[i] = scores_test[0]
#         test_recall_scores[i] = scores_test[1]
#         test_f1_scores[i] = scores_test[2]
        
        
        train_precision_scores[i] = scores_train[0]
        train_recall_scores[i] = scores_train[1]
        train_f1_scores[i] = scores_train[2]
        train_acc_scores[i] = accuracy_score(y_true[train_index], y_pred_train)

        scores_test = precision_recall_fscore_support(y_true[test_index], 
                                                       y_pred_test, 
                                                       average="macro")

        test_precision_scores[i] = scores_test[0]
        test_recall_scores[i] = scores_test[1]
        test_f1_scores[i] = scores_test[2]
        test_acc_scores[i] = accuracy_score(y_true[test_index], y_pred_test)
        
        report = dict(
            runs = cv.get_n_splits(),
            split=i+1,
            train_accuracy = train_acc_scores[i],
            train_precision = train_precision_scores[i],
            train_recall = train_recall_scores[i],
            train_fscore = train_f1_scores[i],
            test_accuracy = test_acc_scores[i],
            test_precision = test_precision_scores[i],
            test_recall = test_recall_scores[i],
            test_fscore = test_f1_scores[i],
        )

        if os.path.exists("data/knn_300_splits.csv"):
            pd.DataFrame([report]).to_csv("data/knn_300_splits.csv", mode="a", header=False, decimal=",", index=False)
        else:
            pd.DataFrame([report]).to_csv("data/knn_300_splits.csv", mode="a", header=True, decimal=",", index=False)
        
#     report = dict(
#         k = k,
#         runs = cv.get_n_splits(),
#         train_precision = train_precision_scores.mean(),
#         train_precision_std = train_precision_scores.std(),
#         train_recall = train_recall_scores.mean(),
#         train_recall_std = train_recall_scores.std(),
#         train_fscore = train_f1_scores.mean(),
#         train_fscore_std = train_f1_scores.std(),
#         test_precision = test_precision_scores.mean(),
#         test_precision_std = test_precision_scores.std(),
#         test_recall = test_recall_scores.mean(),
#         test_recall_std = test_recall_scores.std(),
#         test_fscore = test_f1_scores.mean(),
#         test_fscore_std = test_f1_scores.std(),
        
#     )
    
#     if os.path.exists("data/knn_300_splits.csv"):
#         pd.DataFrame([report]).to_csv("data/knn_300_splits.csv", mode="a", header=False, decimal=",", index=False)
#     else:
#         pd.DataFrame([report]).to_csv("data/knn_300_splits.csv", mode="a", header=True, decimal=",", index=False)
    
        
        
        
#         if test_f1_scores[:i+1].mean() > best_data["f1_scores"].mean():
#             best_data["f1_scores"] = test_f1_scores[:i+1]
#             best_data["k"] = k
            
#             print(f"\nMelhor k: {k}")
#             print(f"Acurácia de treino parcial: {train_acc_scores[:i+1].mean():4} +/- ({train_acc_scores[:i+1].std():4})")
#             print(f"Medida-F de treino parcial: {train_f1_scores[:i+1].mean():4} +/- ({train_f1_scores[:i+1].std():4})\n")

#             print(f"Acurácia de test parcial: {test_acc_scores[:i+1].mean():4} +/- ({test_acc_scores[:i+1].std():4})")
#             print(f"Medida-F de test parcial: {test_f1_scores[:i+1].mean():4} +/- ({test_f1_scores[:i+1].std():4})\n")

    
    

/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass n_splits=10, n_repeats=30, random_state=42 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


[k=3] 1/300 

/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[k=3] 2/300 

/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[k=3] 299/300 

/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[k=3] 300/300 

/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Treinamento com melhor valor de K

- O objetivo aqui é determinar o desempenho do classificador o valor de K com o qual ovtivemos a melhor medida-f média através validação cruzada. Observar os resultados no arquivo **data/knn_classification_report.txt**
- O melhor valor de K pode ser observado no arquivo **data/knn_best_k.csv**, gerado no experimento anterior

In [47]:
from sklearn.metrics import classification_report

BEST_K = 3
#views = [fac, fou, kar]
views = [fac, fou, kar]

train_views = [minmax_scale(v) for v in views]

train_views_dists = [euclidean_distances(v,v) for v in train_views]

train_views_density_probs = [calc_knn_density_prob(v, BEST_K, Pw, y_true) 
                             for v in train_views_dists]

train_views_post_probs = [calc_prob_posteriori(v, Pw) 
                          for v in train_views_density_probs]

y_pred = regra_soma_padrao(train_views_post_probs, Pw)
      
acc_score_knn = accuracy_score(y_true, y_pred)
    
report = classification_report(y_true, y_pred, output_dict=False, digits=4)

with open("data/knn_classification_report.txt", "w") as report_file:
    report_file.write(report)


/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Estimativa pontual e intervalo de confiança

In [48]:
p = acc_score_knn
z_padrao = 1.96 # Confiança de 95%

print("Estimativa pontual: ", p)
print("Intervalo de confiança: ", calc_intervalo_confiança(p, z_padrao))

Estimativa pontual:  0.811
Intervalo de confiança:  (0.7938, 0.8282)


# iii) Janela de Parzen

## Função de densidade

In [49]:
def parzen_density_function(view, h, partition):
    qtd_x = view.shape[0]
    qtd_w = len(partition)
    
    p_x_w = np.empty((qtd_x, qtd_w))
    dims = view.shape[1]
    
    for i in range(qtd_w):
        n = len(partition[i])
        x_view = view[partition[i],:]
        for k in range(qtd_x):
            diff = (view[k] - x_view)/h
            gaussian_kernel = np.exp(-(diff**2)/2)/np.sqrt(2*np.pi)
            prod_dims = gaussian_kernel.prod(axis=1)
            p_x_w[k,i] = prod_dims.sum()/(n*h**dims)
            
    return p_x_w

In [50]:
# fac_parzen_density_probs = parzen_density_function(fac, 2, partition)
# fou_parzen_density_probs = parzen_density_function(fou, 2, partition)
# kar_parzen_density_probs = parzen_density_function(kar, 2, partition)

## Prob. à posteriori por view

In [51]:
# fac_prazen_posteriori_probs = calc_prob_posteriori(fac_parzen_density_probs, Pw)
# fou_prazen_posteriori_probs = calc_prob_posteriori(fou_parzen_density_probs, Pw)
# kar_prazen_posteriori_probs = calc_prob_posteriori(kar_parzen_density_probs, Pw)

## Regra da soma

In [52]:
# for h in range(2, 21, 1):
#     fac_parzen_density_probs = parzen_density_function(fac, h, partition)
#     fou_parzen_density_probs = parzen_density_function(fou, h, partition)
#     kar_parzen_density_probs = parzen_density_function(kar, h, partition)

#     fac_prazen_posteriori_probs = calc_prob_posteriori(fac_parzen_density_probs, Pw)
#     fou_prazen_posteriori_probs = calc_prob_posteriori(fou_parzen_density_probs, Pw)
#     kar_prazen_posteriori_probs = calc_prob_posteriori(kar_parzen_density_probs, Pw)

#     y_pred_prazen_all = regra_soma_padrao([fac_prazen_posteriori_probs, 
#                                        fou_prazen_posteriori_probs, 
#                                        kar_prazen_posteriori_probs], Pw)

#     print(f"h: {h}")
#     print("Acurácia: ", accuracy_score(y_true, y_pred_prazen_all))
#     print("Medida-F: ", f1_score(y_true, y_pred_prazen_all, average="macro"))

## Validação cruzada

- Variamos h de 2 a 12 para fins de comparação. Não foi possíve, na validação cruzada, avaliar valores de janela maiores por limitação de tempo.

In [53]:
cv = RepeatedStratifiedKFold(FOLDS, REPEATS, RANDOM_SEED)
train_acc_scores = np.empty((cv.get_n_splits(),))
train_f1_scores = np.empty((cv.get_n_splits(),))

test_acc_scores = np.empty((cv.get_n_splits(),))
test_f1_scores = np.empty((cv.get_n_splits(),))


h_range = range(2,3)
best_data = {"f1_scores": np.zeros((1,)), 
             "h":None}

views = [fac, fou, kar]

y_true = y_true.astype(int)
for h in h_range:
    train_precision_scores = np.empty((cv.get_n_splits(),))
    train_recall_scores = np.empty((cv.get_n_splits(),))
    train_f1_scores = np.empty((cv.get_n_splits(),))
    train_acc_scores = np.empty((cv.get_n_splits(),))

    test_precision_scores = np.empty((cv.get_n_splits(),))
    test_recall_scores = np.empty((cv.get_n_splits(),))
    test_f1_scores = np.empty((cv.get_n_splits(),))
    test_acc_scores = np.empty((cv.get_n_splits(),))
    
    for i, (train_index, test_index) in enumerate(cv.split(fac, y_true)):
        print(f"[h={h}] {i+1}/{cv.get_n_splits()}", end=" ")

        train_views = [v[train_index] for v in views]
        test_views = [v[test_index] for v in views]

        
        y_true_split = get_splited_partition(train_index, y_true)
        

        train_views_density_probs = [parzen_density_function(v, h, y_true_split) 
                                     for v in train_views]

        train_views_post_probs = [calc_prob_posteriori(v, Pw) 
                                  for v in train_views_density_probs]

        y_pred_train = regra_soma_padrao(train_views_post_probs, Pw)


        y_true_split = get_splited_partition(test_index, y_true)
        
        test_views_density_probs = [parzen_density_function(v, h, y_true_split) 
                                     for v in test_views]

        test_views_post_probs = [calc_prob_posteriori(v, Pw) 
                                  for v in test_views_density_probs]

        y_pred_test = regra_soma_padrao(test_views_post_probs, Pw)

        scores_train = precision_recall_fscore_support(y_true[train_index], 
                                                       y_pred_train, 
                                                       average="macro")

        train_precision_scores[i] = scores_train[0]
        train_recall_scores[i] = scores_train[1]
        train_f1_scores[i] = scores_train[2]
        train_acc_scores[i] = accuracy_score(y_true[train_index], y_pred_train)

        scores_test = precision_recall_fscore_support(y_true[test_index], 
                                                       y_pred_test, 
                                                       average="macro")

        test_precision_scores[i] = scores_test[0]
        test_recall_scores[i] = scores_test[1]
        test_f1_scores[i] = scores_test[2]
        test_acc_scores[i] = accuracy_score(y_true[test_index], y_pred_test)
        
        report = dict(
            runs = cv.get_n_splits(),
            split=i+1,
            train_accuracy = train_acc_scores[i],
            train_precision = train_precision_scores[i],
            train_recall = train_recall_scores[i],
            train_fscore = train_f1_scores[i],
            test_accuracy = test_acc_scores[i],
            test_precision = test_precision_scores[i],
            test_recall = test_recall_scores[i],
            test_fscore = test_f1_scores[i],
        )

        if os.path.exists("data/parzen_300_splits.csv"):
            pd.DataFrame([report]).to_csv("data/parzen_300_splits.csv", mode="a", header=False, decimal=",", index=False)
        else:
            pd.DataFrame([report]).to_csv("data/parzen_300_splits.csv", mode="a", header=True, decimal=",", index=False)

        
#         report = dict(
#             h = h,
#             runs = cv.get_n_splits(),
#             train_precision = train_precision_scores.mean(),
#             train_precision_std = train_precision_scores.std(),
#             train_recall = train_recall_scores.mean(),
#             train_recall_std = train_recall_scores.std(),
#             train_fscore = train_f1_scores.mean(),
#             train_fscore_std = train_f1_scores.std(),
#             test_precision = test_precision_scores.mean(),
#             test_precision_std = test_precision_scores.std(),
#             test_recall = test_recall_scores.mean(),
#             test_recall_std = test_recall_scores.std(),
#             test_fscore = test_f1_scores.mean(),
#             test_fscore_std = test_f1_scores.std(),
#         )

#     if os.path.exists("data/parzen_300_splits.csv"):
#         pd.DataFrame([report]).to_csv("data/parzen_300_splits.csv", mode="a", header=False, decimal=",", index=False)
#     else:
#         pd.DataFrame([report]).to_csv("data/parzen_300_splits.csv", mode="a", header=True, decimal=",", index=False)
    
    

/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass n_splits=10, n_repeats=30, random_state=42 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


[h=2] 1/300 [h=2] 2/300 [h=2] 3/300 [h=2] 4/300 [h=2] 5/300 [h=2] 6/300 [h=2] 7/300 [h=2] 8/300 [h=2] 9/300 [h=2] 10/300 [h=2] 11/300 [h=2] 12/300 [h=2] 13/300 [h=2] 14/300 [h=2] 15/300 [h=2] 16/300 [h=2] 17/300 [h=2] 18/300 [h=2] 19/300 [h=2] 20/300 [h=2] 21/300 [h=2] 22/300 [h=2] 23/300 [h=2] 24/300 [h=2] 25/300 [h=2] 26/300 [h=2] 27/300 [h=2] 28/300 [h=2] 29/300 [h=2] 30/300 [h=2] 31/300 [h=2] 32/300 [h=2] 33/300 [h=2] 34/300 [h=2] 35/300 [h=2] 36/300 [h=2] 37/300 [h=2] 38/300 [h=2] 39/300 [h=2] 40/300 [h=2] 41/300 [h=2] 42/300 [h=2] 43/300 [h=2] 44/300 [h=2] 45/300 [h=2] 46/300 [h=2] 47/300 [h=2] 48/300 [h=2] 49/300 [h=2] 50/300 [h=2] 51/300 [h=2] 52/300 [h=2] 53/300 [h=2] 54/300 [h=2] 55/300 [h=2] 56/300 [h=2] 57/300 [h=2] 58/300 [h=2] 59/300 [h=2] 60/300 [h=2] 61/300 [h=2] 62/300 [h=2] 63/300 [h=2] 64/300 [h=2] 65/300 [h=2] 66/300 [h=2] 67/300 [h=2] 68/300 [h=2] 69/300 [h=2] 70/300 [h=2] 71/300 [h=2] 72/300 [h=2] 73/300 [h=2] 74/300 [h=2] 75/300 [h=2] 76/300 [h=2] 77/300 [h=2] 78

## Treinamento com melhor valor de h

- O objetivo aqui é determinar o desempenho do classificador o valor de h com o qual ovtivemos a melhor medida-f média através validação cruzada. Observar os resultados no arquivo **data/parzen_classification_report_h2.txt**
- Contudo, assim como com a validação cruzada, o modelo obteve 100% em todas as medidas, para todos os valores de h e em ambas as bases de treinamento e validação.

- Valores de h a partir de 30 causaram underflow no trenamento graças à poderação exponecial feita no cálculo da função de densidade do modelo e portanto não puderam ser computadas.

In [54]:
from sklearn.metrics import classification_report

BEST_H = 2
#views = [fac, fou, kar]
views = [fac, fou, kar]

train_views_density_probs = [parzen_density_function(v, BEST_H, partition) 
                             for v in views]

train_views_post_probs = [calc_prob_posteriori(v, Pw) 
                          for v in train_views_density_probs]

y_pred = regra_soma_padrao(train_views_post_probs, Pw)

report = classification_report(y_true, y_pred, output_dict=False, digits=4)

acc_score_parzen = accuracy_score(y_true, y_pred)

with open(f"data/parzen_classification_report_h{BEST_H}.txt", "w") as report_file:
    report_file.write(report)


## Estimativa pontual e intervalo de confiança

In [55]:
p = acc_score_parzen
z_padrao = 1.96 # Confiança de 95%

print("Estimativa pontual: ", p)
print("Intervalo de confiança: ", calc_intervalo_confiança(p, z_padrao))

Estimativa pontual:  1.0
Intervalo de confiança:  (1.0, 1.0)


## Friedman test

In [2]:
from scipy.stats import friedmanchisquare
import pandas as pd

knn_scores = pd.read_csv("data/knn_300_splits.csv", decimal=",")
gaussian_scores = pd.read_csv("data/gaussian_300_splits.csv", decimal=",")
parzen_scores = pd.read_csv("data/parzen_300_splits.csv", decimal=",")

knn_acc = knn_scores["test_accuracy"].values
gaussian_acc = gaussian_scores["test_accuracy"].values
parzen_acc = parzen_scores["test_accuracy"].values

friedmanchisquare(knn_acc, gaussian_acc, parzen_acc)

FriedmanchisquareResult(statistic=513.6526138279932, pvalue=2.895690222219934e-112)

In [3]:
import scikit_posthocs as sp

sp.posthoc_nemenyi_friedman([knn_acc, gaussian_acc, parzen_acc])

Matplotlib is building the font cache; this may take a moment.


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,1.0,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9
1,0.9,1.0,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9
2,0.9,0.9,1.0,0.9,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9
3,0.9,0.9,0.9,1.0,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9
4,0.9,0.9,0.9,0.9,1.0,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,1.0,0.9,0.9,0.9,0.9
296,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,1.0,0.9,0.9,0.9
297,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,1.0,0.9,0.9
298,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,1.0,0.9


In [87]:
",".join(map(str, knn_acc))

'0.745,0.785,0.73,0.72,0.725,0.725,0.72,0.78,0.765,0.745,0.75,0.75,0.805,0.785,0.75,0.81,0.71,0.78,0.795,0.715,0.75,0.74,0.77,0.725,0.77,0.795,0.725,0.745,0.745,0.7,0.695,0.78,0.695,0.725,0.76,0.77,0.72,0.72,0.755,0.75,0.785,0.72,0.77,0.795,0.735,0.765,0.7,0.715,0.715,0.72,0.715,0.79,0.735,0.69,0.745,0.78,0.755,0.755,0.78,0.73,0.72,0.72,0.735,0.78,0.725,0.82,0.755,0.775,0.765,0.75,0.76,0.705,0.76,0.76,0.72,0.76,0.78,0.75,0.77,0.79,0.73,0.715,0.775,0.72,0.76,0.73,0.69,0.755,0.765,0.76,0.68,0.72,0.75,0.785,0.75,0.765,0.73,0.78,0.71,0.8,0.76,0.74,0.775,0.745,0.77,0.725,0.74,0.775,0.77,0.735,0.73,0.725,0.685,0.79,0.75,0.72,0.745,0.755,0.745,0.76,0.75,0.735,0.745,0.71,0.71,0.76,0.765,0.74,0.745,0.72,0.765,0.77,0.725,0.745,0.775,0.725,0.705,0.73,0.78,0.775,0.745,0.735,0.76,0.75,0.795,0.725,0.77,0.705,0.79,0.715,0.73,0.725,0.76,0.7,0.78,0.73,0.785,0.785,0.785,0.715,0.765,0.74,0.76,0.775,0.73,0.73,0.72,0.77,0.765,0.715,0.76,0.75,0.785,0.7,0.79,0.76,0.705,0.78,0.72,0.775,0.775,0.74,0.745,0.78,0

In [88]:
",".join(map(str, gaussian_acc))

'0.715,0.775,0.71,0.73,0.77,0.72,0.705,0.685,0.76,0.665,0.705,0.71,0.75,0.73,0.745,0.755,0.7,0.725,0.76,0.69,0.725,0.75,0.69,0.735,0.72,0.76,0.68,0.705,0.705,0.76,0.7,0.74,0.685,0.695,0.71,0.745,0.73,0.715,0.755,0.725,0.765,0.73,0.745,0.765,0.685,0.705,0.72,0.73,0.72,0.69,0.71,0.725,0.78,0.765,0.695,0.715,0.715,0.71,0.695,0.73,0.665,0.75,0.695,0.705,0.72,0.745,0.755,0.715,0.735,0.745,0.73,0.69,0.73,0.735,0.695,0.735,0.73,0.72,0.725,0.725,0.725,0.765,0.76,0.675,0.715,0.7,0.695,0.745,0.72,0.725,0.725,0.72,0.695,0.725,0.74,0.76,0.685,0.775,0.695,0.73,0.745,0.74,0.73,0.71,0.72,0.715,0.695,0.7,0.755,0.755,0.755,0.675,0.675,0.755,0.73,0.72,0.745,0.745,0.705,0.735,0.72,0.765,0.71,0.755,0.705,0.71,0.73,0.695,0.72,0.705,0.715,0.735,0.735,0.735,0.755,0.655,0.69,0.74,0.735,0.685,0.775,0.695,0.72,0.71,0.74,0.71,0.73,0.715,0.74,0.695,0.68,0.73,0.735,0.63,0.72,0.715,0.76,0.755,0.78,0.73,0.725,0.695,0.695,0.785,0.71,0.73,0.71,0.72,0.735,0.695,0.715,0.71,0.735,0.67,0.72,0.73,0.715,0.775,0.735,0.715,0.

In [89]:
",".join(map(str, parzen_acc))

'1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0